In [ ]:
#ÖN İŞLEMEYE DAİR
"Z-SKORU İLE NUMERİK SÜTUNLARDA AYKIRI DEĞER TESPİTİ VE CLIPPING"
"PRICE SÜTUNU İÇİN LOG DÖNÜŞÜMÜ"
def handle_outliers_zscore(df, threshold=3):
    """
    Sayısal sütunlardaki aykırı değerleri Z-skoru yöntemiyle tespit eder ve yönetir.

    Args:
        df (pd.DataFrame): Veri çerçevesi.
        threshold (float): Z-skoru eşik değeri.

    Returns:
        pd.DataFrame: Aykırı değerleri yönetilmiş veri çerçevesi.
    """

    df_numeric = df.select_dtypes(include=np.number)  # Sadece sayısal sütunlar

    for col in df_numeric.columns:
        if col == 'price':  # Fiyat sütununu ayrıca ele al
            continue

        mean = df[col].mean()
        std = df[col].std()
        z_scores = np.abs((df[col] - mean) / std)
        outlier_indices = np.where(z_scores > threshold)[0]
        num_outliers = len(outlier_indices)

        if num_outliers > 0:
            print(f"Sütun: {col}, Aykırı Değer Sayısı: {num_outliers}")
            # Aykırı değerleri görselleştir (boxplot)
            plt.figure(figsize=(8, 6))
            sns.boxplot(x=df[col])
            plt.title(f"Sütun: {col} - Aykırı Değerler")
            plt.show()

            # Sınırlama (Clipping)
            df[col] = np.where(df[col] > mean + threshold * std, mean + threshold * std, df[col])
            df[col] = np.where(df[col] < mean - threshold * std, mean - threshold * std, df[col])

            print(f"Sütun: {col}, Aykırı Değerler Sınırlandırıldı.")

    # Fiyat sütununu ayrıca ele al (log dönüşümü)
    if 'price' in df.columns:
        print("Fiyat sütununa log dönüşümü uygulanıyor...")
        df['price'] = np.log1p(df['price'])  # log1p, log(1 + x) demektir
        print("Fiyat sütununa log dönüşümü uygulandı.")

    return df

# Aykırı değerleri yönet
df = handle_outliers_zscore(df) 

In [ ]:
"train_test_split() YERİNE ÇAPRAZ DOĞRULAMA"

# Çapraz doğrulama stratejisini tanımla
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) # 5 katlı çapraz doğrulama

# Performansı saklamak için listeler
accuracy_scores = []
classification_reports = []
confusion_matrices = []

# Çapraz doğrulama döngüsü
for fold, (train_index, test_index) in enumerate(cv.split(X, y)):
    print(f"\nFold {fold + 1}")
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Modeli eğit
    model.fit(X_train, y_train)

    # Tahmin yap
    y_pred = model.predict(X_test)

    # Performansı değerlendir
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    accuracy_scores.append(accuracy)
    classification_reports.append(report)
    confusion_matrices.append(cm)

    print(f"Accuracy: {accuracy:.4f}")
    print(report)
    print("Confusion Matrix:")
    print(cm)

# Ortalama performansı yazdır
print("\nOrtalama Performans")
print(f"Ortalama Accuracy: {np.mean(accuracy_scores):.4f}")

In [ ]:
"MODEL HİPERPARAMETRELERİ OPTİMİZASYONU - GridSearchCV"

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold

#Hiperparametre optimizasyonu sırasında kullanılan doğrulama verisi ile modelin nihai 
# performansını değerlendirmek için kullanılan test verisinin tamamen ayrı olduğundan emin olunmalıdır.

# Veri setini eğitim ve test kümelerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42, stratify=y) 

# Model tanımlama
model = SVC(random_state=42)

# Hiperparametre optimizasyonu
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['rbf', 'linear'],
    'gamma': ['scale', 'auto', 0.1, 1]
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=cv, verbose=2,n_jobs=-1)  # Tüm veriyle eğitiyoruz.
grid_search.fit(X_train, y_train)  # Sadece eğitim verisiyle eğitiyoruz
best_model = grid_search.best_estimator_  # En iyi modeli al

print("En iyi parametreler:", grid_search.best_params_)
print("En iyi skor:", grid_search.best_score_)

# Test seti üzerinde değerlendirme
y_pred_test = best_model.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
report_test = classification_report(y_test, y_pred_test)
cm_test = confusion_matrix(y_test, y_pred_test)

print("\nTest Seti Performansı:")
print(f"Accuracy: {accuracy_test:.4f}")
print(report_test)
print("Confusion Matrix:")
print(cm_test)

#Bu kod, en iyi parametreleri bulur ve ekrana yazdırır.
#Modeli bu parametrelerle eğitmek için best_model değişkenini kullanabilirsiniz.


In [ ]:
"ÖZNİTELİK SEÇİMİ YÖNTEMLERİ (RFE, ANOVA, Ki-Kare, Lasso) İLE OPTİMİZASYON"

n_features = 10  # Seçilecek öznitelik sayısı

# RFE (Recursive Feature Elimination)
rfe = RFE(estimator=SVC(kernel='linear', random_state=42), n_features_to_select=n_features)
X_train_rfe = rfe.fit_transform(X_train, y_train)
X_test_rfe = rfe.transform(X_test)
selected_features_rfe = X_train.columns[rfe.support_]

# SelectKBest (ANOVA)
anova = SelectKBest(score_func=f_classif, k=n_features)
X_train_anova = anova.fit_transform(X_train, y_train)
X_test_anova = anova.transform(X_test)
selected_features_anova = X_train.columns[anova.get_support()]

# SelectKBest (Ki-kare) - Sadece kategorik verilerle kullanılabilir
#categorical_cols tanımlı olmalıdır
if len(categorical_cols) > 0:
    # Kategorik sütunları Ki-kare testi için uygun formata dönüştürme (gerekirse)
    # Bu örnekte, kategorik sütunların zaten uygun formatta olduğunu varsayıyoruz.
    X_train_chi2 = X_train[categorical_cols]
    X_test_chi2 = X_test[categorical_cols]

    chi2_selector = SelectKBest(score_func=chi2, k=n_features)
    X_train_chi2_selected = chi2_selector.fit_transform(X_train_chi2, y_train)
    X_test_chi2_selected = chi2_selector.transform(X_test_chi2)
    selected_features_chi2 = X_train_chi2.columns[chi2_selector.get_support()]
else:
    selected_features_chi2 = []
    
# Lasso
lasso = Lasso(alpha=0.1, random_state=42)  # alpha değeri ayarlanabilir
lasso.fit(X_train, y_train)
selected_features_lasso = X_train.columns[lasso.coef_ != 0]
X_train_lasso = X_train[selected_features_lasso]
X_test_lasso = X_test[selected_features_lasso]
#Lasso için alpha parametresi düzenlileştirme gücünü kontrol eder. Farklı alpha değerleri deneyerek en iyi sonucu elde etmeye çalışın.

# 3. Sonuçları Karşılaştırma
print(f"Seçilen En İyi {n_features} Özellik:")
print("RFE:", selected_features_rfe)
print("ANOVA:", selected_features_anova)
print("Lasso:", selected_features_lasso.tolist())
if len(categorical_cols) > 0:
    print("Ki-kare:", selected_features_chi2)

# 4. Performansı Değerlendirme (İsteğe Bağlı)
# Bu bölümde, seçilen özelliklerle bir model eğitip performansını karşılaştırabilirsiniz.
# Örneğin, SVC ile deneyebilirsiniz:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# RFE ile seçilen özelliklerle model eğitimi ve değerlendirme
model_rfe = SVC(random_state=42)
model_rfe.fit(X_train_rfe, y_train)
y_pred_rfe = model_rfe.predict(X_test_rfe)
print("\nRFE Performansı:")
print("Accuracy:", accuracy_score(y_test, y_pred_rfe))
print(classification_report(y_test, y_pred_rfe))

# ANOVA ile seçilen özelliklerle model eğitimi ve değerlendirme
model_anova = SVC(random_state=42)
model_anova.fit(X_train_anova, y_train)
y_pred_anova = model_anova.predict(X_test_anova)
print("\nANOVA Performansı:")
print("Accuracy:", accuracy_score(y_test, y_pred_anova))
print(classification_report(y_test, y_pred_anova))

# Lasso ile seçilen özelliklerle model eğitimi ve değerlendirme
model_lasso = SVC(random_state=42)
model_lasso.fit(X_train_lasso, y_train)
y_pred_lasso = model_lasso.predict(X_test_lasso)
print("\nLasso ile Seçilen Özelliklerle Performans:")
print("Accuracy:", accuracy_score(y_test, y_pred_lasso))
print(classification_report(y_test, y_pred_lasso))

if len(categorical_cols) > 0:
    # Ki-kare ile seçilen özelliklerle model eğitimi ve değerlendirme
    model_chi2 = SVC(random_state=42)
    model_chi2.fit(X_train_chi2_selected, y_train)
    y_pred_chi2 = model_chi2.predict(X_test_chi2_selected)
    print("\nKi-kare Performansı:")
    print("Accuracy:", accuracy_score(y_test, y_pred_chi2))
    print(classification_report(y_test, y_pred_chi2))


In [ ]:
"Veri Dengesizliği Varsa SMOTE ile OverSampling ve Yeni Çapraz Doğrulama Döngüsü"

from sklearn.model_selection import StratifiedKFold  # Çapraz doğrulama için
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE, RandomOverSampler  # Sınıf dengeleme için
import numpy as np

# Çapraz doğrulama stratejisi
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  # Önemli: stratify=y

# Model ve sınıf dengeleme yöntemleri
model = SVC(random_state=42)
smote = SMOTE(random_state=42)
ros = RandomOverSampler(random_state=42)

# Sonuçları saklamak için listeler
smote_results = []
ros_results = []

# Çapraz doğrulama döngüsü
for fold, (train_index, val_index) in enumerate(cv.split(X, y)):  # val_index artık doğrulama indeksi
    print(f"\n--- Katman {fold + 1} ---")

    X_train, X_val = X.iloc[train_index], X.iloc[val_index]  # .iloc kullan
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # *Her katmanda ayrı ayrı* sınıf dengeleme
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

    # Ölçeklendirme (Her katmanda ayrı ayrı)
    scaler_smote = StandardScaler()
    X_train_smote = scaler_smote.fit_transform(X_train_smote)
    X_val_smote = scaler_smote.transform(X_val)  # *Eğitim verisi ölçeklendiricisiyle* doğrulama verisini ölçeklendir

    scaler_ros = StandardScaler()
    X_train_ros = scaler_ros.fit_transform(X_train_ros)
    X_val_ros = scaler_ros.transform(X_val)

    # Modelleri eğitme ve değerlendirme
    model.fit(X_train_smote, y_train_smote)
    y_pred_smote = model.predict(X_val_smote)
    smote_results.append(classification_report(y_val, y_pred_smote, output_dict=True))

    model.fit(X_train_ros, y_train_ros)
    y_pred_ros = model.predict(X_val_ros)
    ros_results.append(classification_report(y_val, y_pred_ros, output_dict=True))

# Ortalama sonuçları hesaplama (örnek olarak F1-skoru)
def average_results(results):
    avg_precision = np.mean([r['macro avg']['precision'] for r in results])
    avg_recall = np.mean([r['macro avg']['recall'] for r in results])
    avg_f1 = np.mean([r['macro avg']['f1-score'] for r in results])
    return avg_precision, avg_recall, avg_f1

avg_smote_precision, avg_smote_recall, avg_smote_f1 = average_results(smote_results)
avg_ros_precision, avg_ros_recall, avg_ros_f1 = average_results(ros_results)

print("\n--- Ortalama Sonuçlar (SMOTE) ---")
print(f"Precision: {avg_smote_precision:.4f}, Recall: {avg_smote_recall:.4f}, F1-skoru: {avg_smote_f1:.4f}")

print("\n--- Ortalama Sonuçlar (RandomOverSampler) ---")
print(f"Precision: {avg_ros_precision:.4f}, Recall: {avg_ros_recall:.4f}, F1-skoru: {avg_ros_f1:.4f}")

 
# En iyi sınıf dengeleme yöntemini belirleme (örnek olarak, en yüksek F1-skoru)
if avg_smote_f1 > avg_ros_f1:
    best_balancer = SMOTE(random_state=42)
    X_train_best, y_train_best = best_balancer.fit_resample(X_train, y_train)
    print("\nSMOTE en iyi dengeleme yöntemi olarak seçildi.")
else:
    best_balancer = RandomOverSampler(random_state=42)
    X_train_best, y_train_best = best_balancer.fit_resample(X_train, y_train)
    print("\nRandomOverSampler en iyi dengeleme yöntemi olarak seçildi.")

# Tüm eğitim verisi üzerinde en iyi yöntemle yeniden dengeleme
X_train_final = scaler.fit_transform(X_train_best)  # Tüm eğitim verisiyle ölçeklendir
X_test_final = scaler.transform(X_test)  # Test verisini eğitim ölçeklendiricisiyle ölçeklendir

# Modeli tüm eğitim verisiyle eğitme
final_model = SVC(random_state=42)  # En iyi hiperparametrelerle (varsa)
final_model.fit(X_train_final, y_train_best)

# Test verisi üzerinde değerlendirme
y_pred_final = final_model.predict(X_test_final)
print("\n--- Nihai Model Performansı (Test Verisi Üzerinde) ---")
print(classification_report(y_test, y_pred_final))

In [ ]:
"Polinomsal Özellikler Üretme"
"Veriye uygun değilse veya işleri çok karmaşıklaştırırsa çıkarılabilir -T"

from sklearn.preprocessing import PolynomialFeatures

# Veri setini eğitim ve test kümelerine ayırdıktan ve ölçeklendirdikten sonra (yukarıdaki kodda)
# PolynomialFeatures uygulaması
poly = PolynomialFeatures(degree=2, include_bias=False)  # degree polinomun derecesi
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

print("Orijinal Özellik Sayısı:", X_train.shape[1])
print("Polinomsal Özellik Sayısı:", X_train_poly.shape[1])

# Modeli polinomsal özelliklerle eğitme
model_poly = SVC(random_state=42)
model_poly.fit(X_train_poly, y_train)
y_pred_poly = model_poly.predict(X_test_poly)

print("\nPolinomsal Özelliklerle Model Performansı:")
print(classification_report(y_test, y_pred_poly))

In [ ]:
"EKSİK VERİLERİ KNN VEYA REGRESYON İLE DOLDURMA"

from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
import pandas as pd

def intelligent_imputation(X_train, X_test, numeric_cols, method='knn', n_neighbors=5):
    """
    Eksik değerleri sayısal sütunlarda akıllı yöntemlerle doldurur.

    Args:
        X_train (pd.DataFrame): Eğitim veri seti.
        X_test (pd.DataFrame): Test veri seti.
        numeric_cols (list): Sayısal sütunların listesi.
        method (str, optional): 'knn' veya 'regression'. Varsayılan 'knn'.
        n_neighbors (int, optional): KNN için komşu sayısı. Varsayılan 5.

    Returns:
        tuple: Eksik değerleri doldurulmuş X_train ve X_test.
    """

    X_train_imputed = X_train.copy()
    X_test_imputed = X_test.copy()

    for col in numeric_cols:
        if X_train[col].isnull().any():  # Sütunda eksik değer varsa
            print(f"\nSütun: {col}")

            if method == 'knn':
                imputer = KNNImputer(n_neighbors=n_neighbors)
                X_train_imputed[[col]] = imputer.fit_transform(X_train[[col]])
                X_test_imputed[[col]] = imputer.transform(X_test[[col]])
                print(f"{col} sütunu için KNN imputation uygulandı.")

            elif method == 'regression':
                X_train_temp = X_train.drop(col, axis=1)
                y_train_temp = X_train[col]

                X_train_not_null = X_train_temp[y_train_temp.notna()]
                y_train_not_null = y_train_temp[y_train_temp.notna()]

                model = LinearRegression()
                model.fit(X_train_not_null, y_train_not_null)

                X_train_null = X_train_temp[y_train_temp.isna()]
                X_test_null = X_test[X_test[col].isna()].drop(col, axis=1, errors='ignore')  # Test setinde sütun olmayabilir

                X_train_imputed.loc[X_train[col].isna(), col] = model.predict(X_train_null)
                if not X_test_null.empty:
                    X_test_imputed.loc[X_test[col].isna(), col] = model.predict(X_test_null)
                print(f"{col} sütunu için regresyon imputation uygulandı.")
            else:
                raise ValueError("Geçersiz method. 'knn' veya 'regression' seçin.")

            print(f"Eksik Değer Sayısı (Eğitim): {X_train_imputed[col].isnull().sum()}")
            print(f"Eksik Değer Sayısı (Test): {X_test_imputed[col].isnull().sum()}")

    return X_train_imputed, X_test_imputed


"Kullanım örneği"
numeric_cols = X_train.select_dtypes(include=np.number).columns.tolist()  # Sayısal sütunları al

X_train_imputed, X_test_imputed = intelligent_imputation(X_train.copy(), X_test.copy(), numeric_cols, method='knn', n_neighbors=5)

# veya

X_train_imputed, X_test_imputed = intelligent_imputation(X_train.copy(), X_test.copy(), numeric_cols, method='regression')

# Artık X_train_imputed ve X_test_imputed'ı model eğitiminde kullanabilirsiniz.

In [ ]:
"PCA İLE BOYUT İNDİRGEME"
"Şart değil -T"
#PCA, özellikle yüksek korelasyonlu özellikler varsa faydalı olabilir.

from sklearn.decomposition import PCA

# Veri setini ölçeklendirdikten sonra (yukarıdaki kodda)
# PCA uygulaması
pca = PCA(n_components=0.95)  # Toplam varyansın %95'ini koruyacak kadar bileşen seç
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

print("Orijinal Özellik Sayısı:", X_train.shape[1])
print("PCA Sonrası Özellik Sayısı:", X_train_pca.shape[1])

# Modeli PCA ile dönüştürülmüş verilerle eğitme
model_pca = SVC(random_state=42)
model_pca.fit(X_train_pca, y_train)
y_pred_pca = model_pca.predict(X_test_pca)

print("\nPCA ile Model Performansı:")
print(classification_report(y_test, y_pred_pca))
